In [ ]:
import os
from os import path
from glob import glob
os.chdir('../')
from lib.preprocess_utils import show_img
import cv2
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import mmcv
import lmdb


In [ ]:
csv1 = pd.read_csv('/mnt/nas4/diskl/MMG/Data/MMG-R1/SAMPLE_DATA_BREAST/breast_lv_train_set.csv')
csv2 = pd.read_csv('/mnt/nas4/diskl/MMG/Data/MMG-R1/SAMPLE_DATA_BREAST/breast_lv_val_set.csv')
csv3 = pd.read_csv('/mnt/nas4/diskl/MMG/Data/MMG-R1/SAMPLE_DATA_BREAST/breast_lv_test_set.csv')

csv_f = pd.concat([csv1, csv2, csv3])

In [ ]:
env = lmdb.open("/mnt/nas4/diskl/MMG/Data/MMG-R1/SAMPLE_DATA_BREAST/csaw_breast_lv_set/", map_size=1099511627776) 
  
with env.begin() as txn:
   myList = [ key[2:-1] for key, _ in txn.cursor() ]
print(myList)

In [ ]:
for idx, row in csv_f.iterrows():
    if row['cc_view'] not in myList:
        print(row['cc_view'])
    if row['mlo_view'] not in myList:
        print(row['mlo_view'])

In [ ]:
img = cv2.imread('/mnt/f/datasets/cbis-ddsm-png/full_mammogram_images/Mass-Training_P_01031_LEFT_CC.png', cv2.IMREAD_ANYDEPTH)
img = ((img/65535).astype(np.float32)-0.4)/0.2
show_img(img)

In [ ]:
env = lmdb.open("/mnt/nas4/diskl/MMG/Data/MMG-R1/SAMPLE_DATA/csaw_set/", map_size=1099511627776) 
  
txn = env.begin(write=True)
img_bytes = txn.get('00020_20990909_R_CC_3.dcm'.encode()) 
img_np = np.frombuffer(img_bytes, np.uint16)
img_np = img_np.reshape((2,896,1120))
show_img(img_np[0])

# print(img_np.min(), img_np.max())
# img_np = ((np.repeat(img_np[:, :, np.newaxis], 3, axis=2)/65535).astype(np.float32)-0.4)/0.2
# print(img_np.min(), img_np.max())
# show_img(img_np)

In [ ]:
from lib.preprocess_utils import read_resize_img, segment_breast, horizontal_flip,convert_to_16bit,remove_pectoral,crop_img,clahe,crop_borders
RESIZE = (1120, 896)

In [ ]:
def process_pipeline(img_file_path):
    target_height, target_width = RESIZE
    img = read_resize_img(img_file_path)
    img = crop_borders(img,border_size=(0,0,0.04,0.04))
    img = np.clip(img,0.1*65535, 0.9*65535)
    img = convert_to_16bit(img)
    img_segment,crop_rect,breast_mask = segment_breast(img)
    breast_mask = crop_img(breast_mask, crop_rect)
    img_segment = remove_pectoral(img=img_segment)
#     img_filped = horizontal_flip(img_segment,breast_mask)
#     img_filped = clahe(img_filped*65535, 65535)
#     img_resized = cv2.resize(img_filped,dsize=(target_width, target_height), 
#             interpolation=cv2.INTER_CUBIC)
#     img_resized = convert_to_16bit(img_resized)
    return img_segment

In [ ]:
img_file = '/mnt/f/datasets/cbis-ddsm-png/full_mammogram_images/Mass-Training_P_00080_RIGHT_MLO.png'

In [ ]:
img = cv2.imread(img_file, cv2.IMREAD_ANYDEPTH)
show_img(img)

In [ ]:
img = process_pipeline(img_file)
print(img.min(),img.max())

show_img(img)


In [ ]:
show_img(torch.squeeze(data_dict['img']))

In [ ]:
data_dict['img'][0]

In [ ]:
hist_full